## Introduction
This notebook is for comparing the time spent of the PCA (Only calculate the first PC using the [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)) versus our approximation method. We use the PC1 created by juicer_tools as the ground truth (GM12878, chromosome 2, resolution 25Kb), which we call it the `juicer_pc1_np`, for comparing the `similar_rate` between tracks (e.g. The Scikit-learn created PC1 versus the `juicer_pc1_np`, and the Approximated PC1-pattern versus the `juicer_pc1_np`). The `similar_rate` is defined as the number of entries in the track1 (e.g. `juicer_pc1_np`) that have a same positive/negative sign as the track2 (e.g. Approximated PC1-pattern) entries compared.

Note that you will have to take a few hours to create the ground truth `juicer_pc1_np` using the juicer_tools in advance, (In our cases, over 4 hours), besides the calculation is only performed on the not `NaN` entries in this notebook.

In summarize, we find that the sign-pattern of the PC1 created by juicer_tools or Scikit-learn are slightly different, the `similar_rate` between the Scikit-learn calculated PC1 versus the Approximated PC1-pattern are higher than the `juicer_pc1_np` versus the Approximated PC1-pattern. Further more, we found that most of the columns in the Hi-C Pearson's covariance matrix are suitable for being selected as the Approximated PC1-pattern, since most of them has a `similar_rate` over 95% versus the ground truth `juicer_pc1_np`, hence we might utilize the random sampling to speed up and reduce the memory requirement of the covariance matrix calculation. 

In [1]:
# Please specify the path for your GM12878, Chromosome 2, 25Kb Pearson matrix & PC1 text file created by juicer_tools.
pearson_path = "./data/gm12878_pearson_25000_chr2.txt"
pc1_path = "./data/gm12878_pc1_25000_chr2.txt" # Ground Truth.

In [2]:
import time
import numpy as np
import pandas as pd
import math
from random import sample
from sklearn.decomposition import PCA
from hicpap import paptools

Define a function to flip the sign of the PC1 or Approximated PC1-pattern according to the reference genome's GC-content distribution. The reference genome we used in this notebook is the GRCh37 (hg19) assembly.

In [3]:
def flip_track_gc(track_np: np.ndarray, gc_np: np.ndarray) -> np.ndarray:
    if np.nanmean(gc_np[track_np[:-1] > 0]) < np.nanmean(gc_np[track_np[:-1] < 0]):
        track_np = -track_np
    return track_np

gc_df = pd.read_table("./data/hg19_gc25000_chr2.txt", skiprows=[0], names=["bin", "GC"])
gc_np = gc_df["GC"].values.flatten()

Read in the juicer_tools created Pearson correlation matrix and remove the `NaN` entries.

In [4]:
pearson_np = paptools.read_pearson(pearson=pearson_path)

if len(pearson_np) != len(pearson_np[0]):
    print("Pearson matrix has a different number of rows and columns")

pearson_np = pearson_np.astype('float64')
diag = np.diag(pearson_np)
diag_valid = ~np.isnan(diag)
ixgrid = np.ix_(diag_valid, diag_valid) # Record the position of the valid sub-matrix. 
pearson_np = pearson_np[ixgrid]
valid_entry_num = len(pearson_np)

has_nan = np.isnan(pearson_np).any()

if has_nan:
    print("NaN entries still exist in the Pearson matrix.")

Read in the juicer_tools created PC1 as the comparison ground truth.

In [5]:
juicer_pc1_df = pd.read_table(pc1_path, header=None)
juicer_pc1_np = juicer_pc1_df.values.flatten()
juicer_pc1_np = flip_track_gc(track_np=juicer_pc1_np, gc_np=gc_np)

Calculate the time spent for PCA if we only calculate the first Principal component through Scikit-learn. 

In [6]:
start = time.time()

pca = PCA(n_components=1)
pca.fit(pearson_np)
# Place back the valid entries to it's origin position in the chromosome.
pc1_np = np.full(len(diag_valid), np.nan)
pc1_np[diag_valid] = pca.components_[0]

end = time.time()
print(f"Time spent for performing PCA through Scikit-learn to get the PC1 (seconds): {end - start}")

Time spent for performing PCA through Scikit-learn to get the PC1 (seconds): 3.9506866931915283


Calculate the time spent for creating the Approximated PC1-pattern if we:
1. Construct the covariance matrix of a centered Pearson correlation matrix.
2. Select the row (same as selecting the column, since the covariance matrix is symmetric) of covariance matrix with the maximum summation of the absolute-value of each entry as the Approximate PC1-pattern (We defined as $CxMax$ in our paper).   

Assume we use $X_{d \times d}$ to denote the centered Pearson correlation matrix ($d$ means the number of bins), then the formula for constructing the covariance matrix will be $\frac{1}{n}XX^{T}$.

In [7]:
start = time.time()

pearson_np -= pearson_np.mean(axis=1, keepdims=True)
n = len(pearson_np[0])
cov_np = pearson_np @ pearson_np.T / n
cov_abs_sum = [(index, np.sum(np.abs(row))) for index, row in enumerate(cov_np)] 
sorted_cov_abs_sum = sorted(cov_abs_sum, key=lambda x: x[1], reverse=True) # Sorted from the maximum to the minimum. 
# Place back the valid entries to it's origin position in the chromosome.
approx_np = np.full(len(diag_valid), np.nan)
approx_np[diag_valid] = cov_np[sorted_cov_abs_sum[0][0]]

end = time.time()
print(f"Time spent for creating the Approximated PC1-pattern by finding the CxMax in the full-covariance matrix (seconds): {end - start}")

Time spent for creating the Approximated PC1-pattern by finding the CxMax in the full-covariance matrix (seconds): 11.006757259368896


Showing the similar_rate between the `juicer_pc1_np` and the Scikit-learn created PC1.

In [8]:
pc1_np = flip_track_gc(track_np=pc1_np, gc_np=gc_np)
similar_info = paptools.calc_similarity(track1_np=juicer_pc1_np, track2_np=pc1_np)

print(f"total_entry_num: {similar_info['total_entry_num']}")
print(f"valid_entry_num: {similar_info['valid_entry_num']}")
print(f"similar_num: {similar_info['similar_num']}")
print(f"similar_rate: {np.round(similar_info['similar_rate'], 3)}")

total_entry_num: 9728
valid_entry_num: 9519
similar_num: 9130
similar_rate: 0.959


Showing the similar_rate between the `juicer_pc1_np` and the Approximated PC1-pattern.

In [9]:
approx_np = flip_track_gc(track_np=approx_np, gc_np=gc_np)
similar_info = paptools.calc_similarity(track1_np=juicer_pc1_np, track2_np=approx_np)

print(f"total_entry_num: {similar_info['total_entry_num']}")
print(f"valid_entry_num: {similar_info['valid_entry_num']}")
print(f"similar_num: {similar_info['similar_num']}")
print(f"similar_rate: {np.round(similar_info['similar_rate'], 3)}")

total_entry_num: 9728
valid_entry_num: 9519
similar_num: 9234
similar_rate: 0.97


Showing the similar_rate between the Scikit learn created PC1 and the Approximated PC1-pattern. 

In [10]:
similar_info = paptools.calc_similarity(track1_np=pc1_np, track2_np=approx_np)

print(f"total_entry_num: {similar_info['total_entry_num']}")
print(f"valid_entry_num: {similar_info['valid_entry_num']}")
print(f"similar_num: {similar_info['similar_num']}")
print(f"similar_rate: {np.round(similar_info['similar_rate'], 3)}")

total_entry_num: 9728
valid_entry_num: 9519
similar_num: 9361
similar_rate: 0.983


Since creating the Approximated PC1-pattern is merely selecting one of the columns of the covariance matrix (i.e. $CxMax$), we compute the `similar_rate` between the `juicer_pc1_np` and each column of the covariance matrix. We found that over 91% of these columns have a `similar_rate` higher than 0.9, over 72% have a `similar_rate` higher than 0.95, but 0% have a `similar_rate` higher than 0.99.

In [11]:
similar_rates_over90 = []
similar_rates_over95 = []
similar_rates_over99 = []

for i in range(len(cov_np)):
    # Place back the valid entries to it's origin position in the chromosome.  
    approx_np = np.full(len(diag_valid), np.nan)
    approx_np[diag_valid] = cov_np[i]
    approx_np = flip_track_gc(track_np=approx_np, gc_np=gc_np)
    similar_info = paptools.calc_similarity(track1_np=juicer_pc1_np, track2_np=approx_np)

    if similar_info["similar_rate"] >= 0.9:
        similar_rates_over90.append(similar_info["similar_rate"])
    
    if similar_info["similar_rate"] >= 0.95:
        similar_rates_over95.append(similar_info["similar_rate"])
    
    if similar_info["similar_rate"] >= 0.99:
        similar_rates_over99.append(similar_info["similar_rate"])

print(f"valid_entry_num: {valid_entry_num}")
print(f"similar_rate over 0.9: {np.round(len(similar_rates_over90) / valid_entry_num * 100, 3)}%")
print(f"similar_rate over 0.95: {np.round(len(similar_rates_over95) / valid_entry_num * 100, 3)}%")
print(f"similar_rate over 0.99: {np.round(len(similar_rates_over99) / valid_entry_num * 100, 3)}%")
print(f"Actual values of similar_rate > 0.9: {similar_rates_over90 + similar_rates_over95 + similar_rates_over99}")

valid_entry_num: 9519
similar_rate over 0.9: 91.879%
similar_rate over 0.95: 72.959%
similar_rate over 0.99: 0.0%
Actual values of similar_rate > 0.9: [0.9505200126063662, 0.9187939909654376, 0.9389641769093392, 0.9429561928774031, 0.9425359806702385, 0.9451623069650174, 0.9494694820884547, 0.9497846412438281, 0.9134362853240887, 0.9643870154427986, 0.9241516966067864, 0.9450572539132261, 0.94904926988129, 0.9489442168294989, 0.945582519172182, 0.9482088454669608, 0.9425359806702385, 0.9373883811324719, 0.9482088454669608, 0.9421157684630739, 0.9488391637777077, 0.9330812060090345, 0.9392793360647127, 0.9288790839373884, 0.9027208740413909, 0.9575585670763735, 0.9608152116818993, 0.9360226914591869, 0.9151171341527471, 0.9358125853556046, 0.9412753440487446, 0.9456875722239731, 0.9485240046223343, 0.9424309276184473, 0.9466330496900935, 0.9492593759848723, 0.9533564450047274, 0.9523059144868159, 0.9539867633154743, 0.9530412858493539, 0.9518857022796512, 0.9529362327975628, 0.952831179

We also compare the `similar_rate` between the the Scikit-learn created PC1 and our Approximated PC1-pattern, over 92% of these columns have a `similar_rate` higher than 0.9, over 84% have a `similar_rate` higher than 0.95, and over 36% have a `similar_rate` higher than 0.99.

In [12]:
similar_rates_over90 = []
similar_rates_over95 = []
similar_rates_over99 = []

for i in range(len(cov_np)):
    # Place back the valid entries to it's origin position in the chromosome.  
    approx_np = np.full(len(diag_valid), np.nan)
    approx_np[diag_valid] = cov_np[i]
    approx_np = flip_track_gc(track_np=approx_np, gc_np=gc_np)
    similar_info = paptools.calc_similarity(track1_np=pc1_np, track2_np=approx_np)

    if similar_info["similar_rate"] >= 0.9:
        similar_rates_over90.append(similar_info["similar_rate"])
    
    if similar_info["similar_rate"] >= 0.95:
        similar_rates_over95.append(similar_info["similar_rate"])

    if similar_info["similar_rate"] >= 0.99:
        similar_rates_over99.append(similar_info["similar_rate"])

print(f"valid_entry_num: {valid_entry_num}")
print(f"similar_rate over 0.9: {np.round(len(similar_rates_over90) / valid_entry_num * 100, 3)}%")
print(f"similar_rate over 0.95: {np.round(len(similar_rates_over95) / valid_entry_num * 100, 3)}%")
print(f"similar_rate over 0.99: {np.round(len(similar_rates_over99) / valid_entry_num * 100, 3)}%")
print(f"Actual values of similar_rate > 0.9: {similar_rates_over90 + similar_rates_over95 + similar_rates_over99}")

valid_entry_num: 9519
similar_rate over 0.9: 92.426%
similar_rate over 0.95: 84.652%
similar_rate over 0.99: 36.338%
Actual values of similar_rate > 0.9: [0.9712154638092236, 0.9354974262002311, 0.9596596281121966, 0.9640718562874252, 0.9636516440802605, 0.9677487131001156, 0.9731064187414644, 0.9746822145183317, 0.9295094022481353, 0.9039815106628848, 0.980250026263263, 0.9395944952200861, 0.9676436600483244, 0.9724761004307175, 0.9725811534825086, 0.9690093497216095, 0.9714255699128059, 0.9640718562874252, 0.9574535140245825, 0.9718457821199706, 0.9636516440802605, 0.9726862065342998, 0.9527261266939805, 0.9595545750604055, 0.9474734741044227, 0.9179535665511083, 0.9614455299926463, 0.9876037398886438, 0.9382288055468011, 0.9173232482403614, 0.9552473999369682, 0.9632314318730959, 0.9691144027734006, 0.9723710473789263, 0.9641769093392163, 0.9698497741359386, 0.9743670553629583, 0.9832965647652064, 0.9790944426935603, 0.983716776972371, 0.9829814056098329, 0.978254018279231, 0.981615

The experiments above illustrate the difference between the PC1 created by different method or package, hence the result of the compartment identification might always be slightly inconsistent using the PC1-based method, which means the a little loss in the `similar_rate` between the `juicer_tools` and our Approximated PC1-pattern might also be acceptable.

Besides, these experiment show that if we randomly select $k$ rows of the Pearson correlation matrix and denote as $V_{k \times d}$, and perform the 'partial' covariance matrix calculation (which means $\frac{1}{n}XV^{T}$), the chance that the partial covariance matrix contains tracks that have a `similar_rate` higher than 0.95 is pretty high, hence it will speed up the calculation for finding a appropriate track as the Approximated PC1-pattern. 

Here we show the time spent for creating the Approximated PC1-Pattern by finding the $CxMax$ in the partial covariance matrix through sampling. Assume that we randomly sample 10% of the rows in the Pearson matrix for constructing the partial covariance matrix. 

In [12]:
start = time.time()

n = len(pearson_np[0])
sample_indexes = sample(list(range(n)), math.floor(n * 0.1))

partial_cov_np = pearson_np @ pearson_np[sample_indexes].T / n
partial_cov_abs_sum = [(index, np.sum(np.abs(row))) for index, row in enumerate(partial_cov_np.T)] 
sorted_partial_cov_abs_sum = sorted(partial_cov_abs_sum, key=lambda x: x[1], reverse=True) # Sorted from the maximum to the minimum 

approx_np = np.full(len(diag_valid), np.nan)
approx_np[diag_valid] = partial_cov_np.T[sorted_partial_cov_abs_sum[0][0]]

end = time.time()
print(f"Time spent for creating the Approximated PC1-pattern by finding the CxMax in the partial-covariance matrix through sampling (seconds): {end - start}")

Time spent for creating the Approximated PC1-pattern by finding the CxMax in the partial-covariance matrix through sampling (seconds): 3.4066720008850098


Finally we compare the similar_rate between the `juicer_pc1_np` and the Approximated PC1-pattern created by finding the $CxMax$ in the partial-covariance matrix through sampling.

In [13]:
approx_np = flip_track_gc(track_np=approx_np, gc_np=gc_np)
similar_info = paptools.calc_similarity(track1_np=juicer_pc1_np, track2_np=approx_np)

print(f"total_entry_num: {similar_info['total_entry_num']}")
print(f"valid_entry_num: {similar_info['valid_entry_num']}")
print(f"similar_num: {similar_info['similar_num']}")
print(f"similar_rate: {np.round(similar_info['similar_rate'], 3)}")

total_entry_num: 9728
valid_entry_num: 9519
similar_num: 9234
similar_rate: 0.97
